In [13]:
import duckdb as ddb
import pandas as pd

In [14]:
import os
import sys

# This adds the 'src' directory to the Python path.
src_rel_path = '../src'
src_abs_path = os.path.abspath(src_rel_path)

if src_abs_path not in sys.path:
    sys.path.append(src_abs_path)
    print(f"Added '{src_abs_path}' to sys.path")
else:
    print(f"'{src_abs_path}' is already in sys.path")

'/Users/shubxam/Code/Nifty-500-Live-Sentiment-Analysis/src' is already in sys.path


## check primary key constraints in tables

In [20]:
con = ddb.connect('../datasets/ticker_data.db')

In [21]:
con.execute(
    "SELECT constraint_name FROM information_schema.table_constraints WHERE table_name = 'article_data' AND constraint_type = 'PRIMARY KEY'"
).fetch_df()

,constraint_name
0,article_data_ticker_headline_pkey


In [ ]:
# drop the primary key constraint
con.execute(
    'ALTER TABLE article_data DROP CONSTRAINT IF EXISTS article_data_ticker_headline_date_posted_pkey;'
).fetch_df()

NotImplementedException: Not implemented Error: No support for that ALTER TABLE option yet!

In [17]:
# add the new primary key constraint
con.execute('ALTER TABLE article_data ADD PRIMARY KEY (ticker, headline);')

In [18]:
# commit the changes
con.commit()

In [22]:
con.close()

## compute sentiment scores for all articles in db

In [14]:
import duckdb as ddb

In [ ]:
db_path = '../datasets/ticker_data.db'
conn = ddb.connect(db_path)

In [ ]:
articles = conn.execute('select ticker, headline from article_data_temp').fetch_df()
articles.__len__()

In [19]:
from tqdm import tqdm
from transformers.models.bert import BertForSequenceClassification, BertTokenizer
from transformers.pipelines import pipeline

In [20]:
headlines = articles.headline.tolist()

In [21]:
finbert_1 = BertForSequenceClassification.from_pretrained(
    'yiyanghkust/finbert-tone',
    num_labels=3,
    use_safetensors=True,  # Use safe tensors
)

tokenizer_1 = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

# set top_k=1 to get the most likely label or top_k=None to get all labels
# device=-1 means CPU
nlp_1 = pipeline(
    'sentiment-analysis',
    model=finbert_1,
    tokenizer=tokenizer_1,
    device='mps',
    top_k=None,
    framework='pt',
)

Device set to use mps


In [22]:
results = nlp_1(headlines, batch_size=2048)

In [29]:
flattened_data: list[dict[str, float]] = []

for news_item_sentiment_list in tqdm(iterable=results, desc='Processing Sentiment'):
    news_item_sentiment_dict = {}
    for individual_label_dict in news_item_sentiment_list:
        news_item_sentiment_dict[individual_label_dict['label']] = (
            individual_label_dict['score']
        )
    flattened_data.append(news_item_sentiment_dict)

Processing Sentiment: 100%|██████████| 14623/14623 [00:00<00:00, 411508.65it/s]


In [31]:
df = pd.DataFrame(flattened_data)
df.fillna(0, inplace=True)  # Fill NaN values with 0

In [35]:
df = df.astype(
    {
        'Positive': 'float32',
        'Neutral': 'float32',
        'Negative': 'float32',
    }
)

In [36]:
df

,Neutral,Negative,Positive
0,0.999082,0.000854,6.357017e-05
1,0.897181,0.097918,4.900872e-03
2,0.999982,0.000017,4.303088e-07
3,0.999265,0.000711,2.424426e-05
4,0.999359,0.000159,4.819799e-04
...,...,...,...
14618,0.996375,0.003199,4.263582e-04
14619,0.000017,0.000005,9.999781e-01
14620,0.999989,0.000008,2.886356e-06
14621,0.999980,0.000009,1.117425e-05


## merge the sentiment data with the original articles dataframe

In [41]:
df = pd.merge(
    articles,
    df,
    left_index=True,
    right_index=True,
)

In [43]:
df.loc[:, 'compound'] = (
    df.loc[:, 'Positive']
    .where(df['Positive'] > df['Negative'], -df['Negative'])
    .astype(float)
    .round(4)
)

In [50]:
df.rename(
    columns={
        'Positive': 'positive_sentiment',
        'Negative': 'negative_sentiment',
        'Neutral': 'neutral_sentiment',
        'compound': 'compound_sentiment',
    },
    inplace=True,
)

In [57]:
df

,ticker,headline,neutral_sentiment,negative_sentiment,positive_sentiment,compound_sentiment
0,ADANIENT,"Top Gainers and Losers today on 28 August, 202...",0.999082,0.000854,6.357017e-05,-0.0009
1,ADANIENT,"6 Adani stocks, LIC, and 3 PSUs slip below 20 ...",0.897181,0.097918,4.900872e-03,-0.0979
2,ADANIENT,"Adani Group invests Rs 2,000 crore in Dharavi ...",0.999982,0.000017,4.303088e-07,-0.0000
3,ADANIENT,Adani’s airport business to undergo de-merger ...,0.999265,0.000711,2.424426e-05,-0.0007
4,ADANIENT,Adani Ports acquires 80% stake in global OSV o...,0.999359,0.000159,4.819799e-04,0.0005
...,...,...,...,...,...,...
14618,TECHM,Tech Mahindra Ltd,0.996375,0.003199,4.263582e-04,-0.0032
14619,TITAN,Best Premium Wrist Watches: The Timekeepers Of...,0.000017,0.000005,9.999781e-01,1.0000
14620,ULTRACEMCO,"Stocks to watch: Infosys, Jio Financial, Ultra...",0.999989,0.000008,2.886356e-06,-0.0000
14621,WIPRO,Wipro announces results for the Quarter and Ye...,0.999980,0.000009,1.117425e-05,0.0000


## update the article_data table with the new sentiment data

In [58]:
conn.register('df_virtual', df)

In [59]:
query = """
SELECT 
  adt.ticker, 
  adt.headline, 
  adt.date_posted, 
  adt.source, 
  adt.article_link, 
  dfv.neutral_sentiment, 
  dfv.negative_sentiment, 
  dfv.positive_sentiment, 
  dfv.compound_sentiment, 
  adt.created_at
FROM article_data_temp adt
INNER JOIN df_virtual dfv ON adt.ticker = dfv.ticker
                     AND adt.headline = dfv.headline;
"""

In [60]:
merged_df_from_duckdb = conn.execute(query).fetchdf()

In [61]:
merged_df_from_duckdb

,ticker,headline,date_posted,source,article_link,neutral_sentiment,negative_sentiment,positive_sentiment,compound_sentiment,created_at
0,ADANIENT,"Top Gainers and Losers today on 28 August, 202...",2024-08-29 01:30:14,Livemint,https://www.livemint.com/market/stock-market-n...,0.999082,0.000854,6.357017e-05,-0.0009,2024-08-31 01:30:45.048
1,ADANIENT,"6 Adani stocks, LIC, and 3 PSUs slip below 20 ...",2024-08-29 01:30:14,The Economic Times,https://m.economictimes.com/markets/stocks/new...,0.897181,0.097918,4.900872e-03,-0.0979,2024-08-31 01:30:45.048
2,ADANIENT,"Adani Group invests Rs 2,000 crore in Dharavi ...",2024-08-30 11:30:14,Business Standard,https://www.business-standard.com/companies/ne...,0.999982,0.000017,4.303088e-07,-0.0000,2024-08-31 01:30:45.048
3,ADANIENT,Adani’s airport business to undergo de-merger ...,2024-08-30 01:30:14,Times of India,https://timesofindia.indiatimes.com/business/i...,0.999265,0.000711,2.424426e-05,-0.0007,2024-08-31 01:30:45.048
4,ADANIENT,Adani Ports acquires 80% stake in global OSV o...,2024-08-30 15:30:14,The Financial Express,https://www.financialexpress.com/business/indu...,0.999359,0.000159,4.819799e-04,0.0005,2024-08-31 01:30:45.048
...,...,...,...,...,...,...,...,...,...,...
14618,TECHM,Tech Mahindra Ltd,2024-04-19 11:35:30,Business Standard,https://www.business-standard.com/markets/tech...,0.996375,0.003199,4.263582e-04,-0.0032,2025-04-19 11:35:50.926
14619,TITAN,Best Premium Wrist Watches: The Timekeepers Of...,2025-04-19 05:35:30,Jagran English,https://english.jagran.com/smart-guide/style-v...,0.000017,0.000005,9.999781e-01,1.0000,2025-04-19 11:35:50.926
14620,ULTRACEMCO,"Stocks to watch: Infosys, Jio Financial, Ultra...",2025-04-17 11:35:31,Mint,https://www.livemint.com/market/stock-market-n...,0.999989,0.000008,2.886356e-06,-0.0000,2025-04-19 11:35:50.926
14621,WIPRO,Wipro announces results for the Quarter and Ye...,2025-04-05 11:35:31,Wipro,https://www.wipro.com/newsroom/press-releases/...,0.999980,0.000009,1.117425e-05,0.0000,2025-04-19 11:35:50.926


In [63]:
conn.sql(
    'CREATE OR REPLACE TABLE article_data_temp AS SELECT * FROM merged_df_from_duckdb;'
)

In [64]:
conn.execute('SELECT * FROM article_data_temp LIMIT 5').fetch_df()

,ticker,headline,date_posted,source,article_link,neutral_sentiment,negative_sentiment,positive_sentiment,compound_sentiment,created_at
0,ADANIENT,"Top Gainers and Losers today on 28 August, 202...",2024-08-29 01:30:14,Livemint,https://www.livemint.com/market/stock-market-n...,0.999082,0.000854,6.357017e-05,-0.0009,2024-08-31 01:30:45.048
1,ADANIENT,"6 Adani stocks, LIC, and 3 PSUs slip below 20 ...",2024-08-29 01:30:14,The Economic Times,https://m.economictimes.com/markets/stocks/new...,0.897181,0.097918,4.900872e-03,-0.0979,2024-08-31 01:30:45.048
2,ADANIENT,"Adani Group invests Rs 2,000 crore in Dharavi ...",2024-08-30 11:30:14,Business Standard,https://www.business-standard.com/companies/ne...,0.999982,0.000017,4.303088e-07,-0.0000,2024-08-31 01:30:45.048
3,ADANIENT,Adani’s airport business to undergo de-merger ...,2024-08-30 01:30:14,Times of India,https://timesofindia.indiatimes.com/business/i...,0.999265,0.000711,2.424426e-05,-0.0007,2024-08-31 01:30:45.048
4,ADANIENT,Adani Ports acquires 80% stake in global OSV o...,2024-08-30 15:30:14,The Financial Express,https://www.financialexpress.com/business/indu...,0.999359,0.000159,4.819799e-04,0.0005,2024-08-31 01:30:45.048


In [65]:
conn.close()

# adding existing data to table to check upsert operation

In [39]:
import pandas as pd

from database import DatabaseManager

In [40]:
dbm = DatabaseManager()

In [41]:
articles = dbm.get_articles()
articles.drop(columns=['created_at'], inplace=True)
articles.head()

,ticker,headline,date_posted,source,article_link,neutral_sentiment,negative_sentiment,positive_sentiment,compound_sentiment
0,HDFCBANK,HDFC Bank Q4 Results Live: Profit seen up 3.3%...,2025-04-19 11:32:21,CNBC TV18,https://www.cnbctv18.com/market/hdfc-bank-q4-r...,0.000025,9.484177e-07,9.999737e-01,1.0000
1,ONGC,Market Focus Still On Tariffs; Tesla Leads Key...,2025-04-19 11:15:26,Investor's Business Daily,https://www.investors.com/market-trend/stock-m...,0.989872,2.959660e-03,7.168587e-03,0.0072
2,RELIANCE,Jio Financial Share Price: Q4 Results 2025 Out...,2025-04-19 08:35:27,Jobaaj Stories,https://stories.jobaaj.com/news-updates/market...,0.999996,2.762954e-06,1.634307e-06,-0.0000
3,JIOFIN,Jio Financial Share Price: Q4 Results 2025 Out...,2025-04-19 08:35:24,Jobaaj Stories,https://stories.jobaaj.com/news-updates/market...,0.999996,2.762954e-06,1.634307e-06,-0.0000
4,RELIANCE,Jio Financial Services: FY 2025 dividend incom...,2025-04-19 07:35:27,ET Now,https://www.etnownews.com/markets/jio-financia...,0.999993,6.710098e-06,2.585766e-07,-0.0000


In [42]:
dbm.insert_articles(articles, has_sentiment=True)

2025-04-21 23:53:18.854 | INFO     | database:insert_articles:141 - Inserting articles with sentiment scores for 20 news articles


In [43]:
articles = dbm.get_articles(n=21)
articles

,ticker,headline,date_posted,source,article_link,neutral_sentiment,negative_sentiment,positive_sentiment,compound_sentiment,created_at
0,HDFCBANK,HDFC Bank Q4 Results Live: Profit seen up 3.3%...,2025-04-19 11:32:21,CNBC TV18,https://www.cnbctv18.com/market/hdfc-bank-q4-r...,2.543604e-05,9.484177e-07,9.999737e-01,1.0000,2025-04-21 23:53:18.855
1,ONGC,Market Focus Still On Tariffs; Tesla Leads Key...,2025-04-19 11:15:26,Investor's Business Daily,https://www.investors.com/market-trend/stock-m...,9.898718e-01,2.959660e-03,7.168587e-03,0.0072,2025-04-21 23:53:18.855
2,RELIANCE,Jio Financial Share Price: Q4 Results 2025 Out...,2025-04-19 08:35:27,Jobaaj Stories,https://stories.jobaaj.com/news-updates/market...,9.999956e-01,2.762954e-06,1.634307e-06,-0.0000,2025-04-21 23:53:18.855
3,JIOFIN,Jio Financial Share Price: Q4 Results 2025 Out...,2025-04-19 08:35:24,Jobaaj Stories,https://stories.jobaaj.com/news-updates/market...,9.999956e-01,2.762954e-06,1.634307e-06,-0.0000,2025-04-21 23:53:18.855
4,RELIANCE,Jio Financial Services: FY 2025 dividend incom...,2025-04-19 07:35:27,ET Now,https://www.etnownews.com/markets/jio-financia...,9.999931e-01,6.710098e-06,2.585766e-07,-0.0000,2025-04-21 23:53:18.855
5,JIOFIN,Jio Financial Services: FY 2025 dividend incom...,2025-04-19 07:35:24,ET Now,https://www.etnownews.com/markets/jio-financia...,9.999931e-01,6.710098e-06,2.585766e-07,-0.0000,2025-04-21 23:53:18.855
6,HCLTECH,"Q4 Earnings Next Week: Reliance, HUL, Maruti, ...",2025-04-19 07:35:20,Republic World,https://www.republicworld.com/business/q4-earn...,9.999927e-01,6.483144e-06,8.144183e-07,-0.0000,2025-04-21 23:53:18.855
7,BEL,Bharat Electronics shares get BUY call for 6 m...,2025-04-19 06:35:18,ET Now,https://www.etnownews.com/markets/bharat-elect...,4.348312e-06,8.704934e-08,9.999956e-01,1.0000,2025-04-21 23:53:18.855
8,TITAN,Best Premium Wrist Watches: The Timekeepers Of...,2025-04-19 05:35:30,Jagran English,https://english.jagran.com/smart-guide/style-v...,1.689028e-05,4.988682e-06,9.999781e-01,1.0000,2025-04-21 23:53:18.855
9,ADANIENT,Adani’s 42% Revenue Share Bid Enables Govt To ...,2025-04-19 05:35:14,Goodreturns,https://www.goodreturns.in/news/adani-s-42-rev...,9.994743e-01,1.973229e-05,5.058589e-04,0.0005,2025-04-21 23:53:18.855
